# AFL Data Scrape from afltables.com

In [420]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

## Getting all unique player urls from teams

In [70]:
# How teams appear in team url
team_url_list = ['adelaide', 'brisbanel', 'carlton', 'collingwood', 'essendon', 'fremantle', 
                 'geelong', 'goldcoast', 'gws', 'hawthorn', 'melbourne', 'kangaroos', 'padelaide',
                 'richmond', 'stkilda', 'swans', 'westcoast', 'bullldogs']

In [71]:
player_url_set = set()
for team in team_url_list:
    print(team)
    page = requests.get('https://afltables.com/afl/stats/teams/' + team + '.html')
    soup = BeautifulSoup(page.text)
    tables = soup.findAll("table")
    print('team tables count:', len(tables))
    print()
    if not len(tables):
        raise Exception('Missing Table') 
    for row in tables[0].tbody.findAll('tr'):
        try:
            href = row.find('a').get("href")
        except:
            href = None
            print('Error with row:')
            print(row)
            print()
        if not href:
            raise Exception('Missing href')
        player_url_set.add(href)

adelaide
team tables count: 2

brisbanel
team tables count: 2

carlton
team tables count: 2

collingwood
team tables count: 2

essendon
team tables count: 2

fremantle
team tables count: 2

geelong
team tables count: 2

goldcoast
team tables count: 2

gws
team tables count: 2

hawthorn
team tables count: 2

melbourne
team tables count: 2

kangaroos
team tables count: 2

padelaide
team tables count: 2

richmond
team tables count: 2

stkilda
team tables count: 2

swans
team tables count: 2

westcoast
team tables count: 2

bullldogs
team tables count: 2



In [73]:
print(len(player_url_set))
player_url_set

5475


{'../players/R/Ron_Andrews.html',
 '../players/T/Ted_Whitten0.html',
 '../players/P/Paul_Constance.html',
 '../players/J/James_Madden.html',
 '../players/P/Peter_Cloke.html',
 '../players/J/John_Letcher.html',
 '../players/J/Joshua_Mail.html',
 '../players/J/Jarrod_Lienert.html',
 '../players/C/Cameron_Hunter.html',
 '../players/M/Michael_Browne.html',
 '../players/N/Nathan_Lovett-Murray.html',
 '../players/G/Graham_Hunnibell.html',
 '../players/S/Shane_Kersten.html',
 '../players/A/Aaron_Lord.html',
 '../players/J/John_Durnan.html',
 '../players/M/Mitch_Hallahan.html',
 '../players/J/Jarryn_Geary.html',
 '../players/R/Ray_Boyanich.html',
 '../players/B/Brenton_Phillips.html',
 '../players/B/Beau_Maister.html',
 '../players/J/John_Peter-Budge.html',
 '../players/B/Ben_Ingleton.html',
 '../players/I/Ian_Callinan.html',
 '../players/D/Dylan_McLaren.html',
 '../players/P/Paul_Temay.html',
 '../players/E/Ed_Barlow.html',
 '../players/L/Luke_Darcy.html',
 '../players/J/Jack_Lonie.html',
 '.

## Getting all player data from each player url

In [75]:
players_data_list = []
for i, player_url in enumerate(player_url_set):
    print(i, player_url)
    tables_list = pd.read_html('https://afltables.com/afl/stats' + player_url[2:])
    try:
        season_tables = [table for table in tables_list if isinstance(table.columns, pd.MultiIndex)]
    except Exception as e:
        print(e)
        print()
    if not season_tables:
        print('season table missing')
    print(len(season_tables))
    players_data_list.append((player_url, season_tables))

../players/R/Ron_Andrews.html
../players/T/Ted_Whitten0.html
../players/P/Paul_Constance.html
../players/J/James_Madden.html
../players/P/Peter_Cloke.html
../players/J/John_Letcher.html
../players/J/Joshua_Mail.html
../players/J/Jarrod_Lienert.html
../players/C/Cameron_Hunter.html
../players/M/Michael_Browne.html
../players/N/Nathan_Lovett-Murray.html
../players/G/Graham_Hunnibell.html
../players/S/Shane_Kersten.html
../players/A/Aaron_Lord.html
../players/J/John_Durnan.html
../players/M/Mitch_Hallahan.html
../players/J/Jarryn_Geary.html
../players/R/Ray_Boyanich.html
../players/B/Brenton_Phillips.html
../players/B/Beau_Maister.html
../players/J/John_Peter-Budge.html
../players/B/Ben_Ingleton.html
../players/I/Ian_Callinan.html
../players/D/Dylan_McLaren.html
../players/P/Paul_Temay.html
../players/E/Ed_Barlow.html
../players/L/Luke_Darcy.html
../players/J/Jack_Lonie.html
../players/D/David_Miller.html
../players/I/Irving_Mosquito.html
../players/M/Mathew_Mahoney.html
../players/A/Andr

../players/D/Don_Brookes.html
../players/M/Michael_Nugent.html
../players/T/Tory_Dickson.html
../players/J/Jack_Carroll1.html
../players/K/Kieren_Jack.html
../players/B/Brendon_Ah_Chee.html
../players/D/Des_English.html
../players/M/Matthew_Lobbe.html
../players/M/Michael_Martin.html
../players/D/Dale_Woodhall.html
../players/B/Barry_Richardson.html
../players/J/Jacob_Anstey.html
../players/R/Robert_DiPierdomenico.html
../players/M/Merv_Keane.html
../players/D/Dave_Leng.html
../players/J/Josh_Carmichael.html
../players/M/Mark_McGough.html
../players/B/Billie_Smedts.html
../players/G/Greg_Dear.html
../players/A/Andrew_Crowell.html
../players/T/Tony_Hirst.html
../players/C/Chris_Johnson0.html
../players/D/Damian_Bourke.html
../players/B/Bruce_Greenhill.html
../players/G/Gordon_Polson.html
../players/J/Justin_Cicolella.html
../players/B/Barry_Rowlings.html
../players/T/Tim_Watson.html
../players/B/Brett_James.html
../players/M/Matt_Hannebery.html
../players/W/Wennie_van_Lint.html
../playe

../players/B/Bernie_Conlen.html
../players/I/Ian_Morrison.html
../players/P/Peter_Ruscuklic.html
../players/S/Shane_Cable.html
../players/A/Alan_Hunter.html
../players/T/Trent_West.html
../players/P/Phil_Doherty.html
../players/P/Peter_Robinson.html
../players/D/Darren_Walsh.html
../players/J/Josh_Kennedy1.html
../players/P/Paul_Hawke.html
../players/D/Dave_McGlashan.html
../players/M/Mick_Malthouse.html
../players/J/Jarrad_Redden.html
../players/B/Brent_Piltz.html
../players/B/Brian_McKenzie.html
../players/S/Stephen_Coniglio.html
../players/J/Josh_Tynan.html
../players/S/Shane_Kerrison.html
../players/C/Colin_Alexander.html
../players/D/Darren_Forssman.html
../players/J/Josh_Drummond.html
../players/T/Tony_Francis.html
../players/J/John_Brimacombe.html
../players/B/Ben_McEvoy.html
../players/B/Ben_McKay.html
../players/J/James_Parsons.html
../players/T/Tim_Golds.html
../players/K/Karl_Amon.html
../players/N/Nic_Naitanui.html
../players/P/Peter_Sartori.html
../players/S/Steven_Taubert

../players/P/Phillip_Read.html
../players/D/David_Fanning.html
../players/T/Todd_Breman.html
../players/M/Mark_Bayes.html
../players/R/Robert_Elliott.html
../players/N/Nathan_Jones.html
../players/T/Troy_Taylor.html
../players/S/Steven_Da_Rui.html
../players/A/Ahmed_Saad.html
../players/T/Tony_Armstrong.html
../players/N/Norm_Bussell.html
../players/M/Michael_Byrne.html
../players/D/Daniel_Wulf.html
../players/B/Ben_Howlett.html
../players/L/Lin_Jong.html
../players/B/Barry_Teague.html
../players/R/Ray_Walker.html
../players/T/Tristan_Cartledge.html
../players/V/Val_Perovic.html
../players/M/Mick_Evans.html
../players/A/Adrian_Gallagher.html
../players/T/Tim_Hargreaves.html
../players/B/Brett_Backwell.html
../players/J/Jed_Lamb.html
../players/T/Tom_Gilligan0.html
../players/J/Joel_Hamling.html
../players/T/Todd_Ridley.html
../players/H/Haydon_Kilmartin.html
../players/R/Ryan_Gardner.html
../players/D/David_Gallagher.html
../players/P/Peter_Featherby.html
../players/A/Adam_Thomson.html

../players/N/Neil_Erasmus.html
../players/C/Craig_Bradley.html
../players/M/Michael_Howard.html
../players/W/Will_Langford.html
../players/D/Daniel_Archer.html
../players/M/Michael_Phyland.html
../players/M/Mark_Bunn.html
../players/M/Michael_Gallagher.html
../players/A/Adam_Tomlinson.html
../players/S/Stephen_Newport.html
../players/T/Tom_Sheridan.html
../players/J/Jarrad_Irons.html
../players/T/Tom_Roach.html
../players/P/Peter_Vardy.html
../players/J/John_Hamilton1.html
../players/B/Ben_Allan.html
../players/D/David_Sullivan.html
../players/R/Ross_Dillon.html
../players/S/Steve_Parsons.html
../players/L/Luke_Tapscott.html
../players/P/Peter_Thorne.html
../players/D/Dylan_van_Unen.html
../players/P/Paul_Couch.html
../players/S/Shane_Robertson0.html
../players/C/Craig_Evans.html
../players/D/David_Myers.html
../players/L/Luke_Weller.html
../players/M/Michael_Schulze.html
../players/D/David_Glascott.html
../players/D/David_Wirrpanda.html
../players/L/Lukas_Webb.html
../players/C/Chris_

...

../players/D/Damien_Adkins.html
../players/S/Stephen_Roberts.html
../players/A/Aaron_Black0.html
../players/E/Enrico_Misso.html
../players/A/Andrew_Lukas.html
../players/M/Michael_Kol.html
../players/A/Anton_Grbac.html
../players/M/Matt_Suckling.html
../players/N/Neil_Busse.html
../players/M/Michael_Oaten.html
../players/B/Brad_Hartman.html
../players/G/George_Hewett.html
../players/B/Brayden_Maynard.html
../players/D/David_Kernahan.html
../players/I/Ian_Scott.html
../players/R/Ray_Sampson.html
../players/G/Gordon_Casey.html
../players/S/Sean_Wellman.html
../players/D/David_Hille.html
../players/B/Brendan_Krummel.html
../players/B/Bryce_Retzlaff.html
../players/J/Jordan_de_Goey.html
../players/T/Tony_Elshaug.html
../players/B/Brett_Johnson.html
../players/K/Kevin_Heath.html
../players/B/Ben_Reid.html
../players/C/Con_Gorozidis.html
../players/D/Daryl_Cunningham.html
../players/D/David_Ackerly.html
../players/D/Dylan_Smith.html
../players/P/Paul_Maher.html
../players/J/Jeremy_Smith.html

../players/B/Brodie_Moles.html
../players/K/Kelvin_Steel.html
../players/R/Russell_Dickson1.html
../players/R/Richard_Murrie.html
../players/A/Andrew_Purser.html
../players/L/Luke_Toia.html
../players/D/David_Clarke0.html
../players/M/Matthew_Robbins.html
../players/G/Glenn_Freeborn.html
../players/M/Max_Parker.html
../players/D/Dayne_Beams.html
../players/L/Lee_Walker.html
../players/A/Adam_Contessa.html
../players/B/Barry_Mugeli.html
../players/P/Paul_Tolson.html
../players/G/Gary_Moorcroft.html
../players/J/Jake_Aarts.html
../players/K/Kayle_Kirby.html
../players/M/Mark_Cullen.html
../players/S/Sam_Wicks.html
../players/B/Brandon_Walker.html
../players/L/Lachlan_Bramble.html
../players/W/Will_Hoskin-Elliott.html
../players/G/Gavin_Bayes.html
../players/B/Bailey_Rice.html
../players/S/Scott_Lee.html
../players/N/Nakia_Cockatoo.html
../players/G/Graham_Osborne0.html
../players/S/Sam_Darley.html
../players/D/Des_Ryan.html
../players/Z/Zac_Clarke.html
../players/M/Matthew_Ball.html
../p

../players/L/Luke_McPharlin.html
../players/D/Denis_Lenaghan.html
../players/J/Justin_Clarke.html
../players/M/Mitch_Lewis.html
../players/C/Charlie_Payne.html
../players/M/Matthew_Wright.html
../players/T/Trent_Bartlett.html
../players/R/Ray_Card.html
../players/J/Jeff_Hilton.html
../players/C/Clint_Bizzell.html
../players/C/Chad_Morrison.html
../players/S/Shane_Crothers.html
../players/C/Craig_Smoker.html
../players/P/Paul_Stewart.html
../players/T/Terry_Wilkins.html
../players/A/Archie_Perkins.html
../players/E/Eric_Leech.html
../players/A/Allen_Daniels.html
../players/J/Jason_Akermanis.html
../players/A/Alik_Magin.html
../players/G/Greg_Parke.html
../players/B/Bob_Murray.html
../players/D/Danny_Dickfos.html
../players/C/Clancee_Pearce.html
../players/I/Ian_Rickman.html
../players/N/Nathan_Djerrkura.html
../players/B/Brendon_Gale.html
../players/K/Kris_Barlow.html
../players/J/Jack_Payne.html
../players/J/Jordon_Sweet.html
../players/F/Frank_Gumbleton.html
../players/J/Josh_Schoenfe

../players/T/Tom_Logan.html
../players/B/Brent_Daniels.html
../players/J/Jack_Clarke.html
../players/D/Daniel_McPherson.html
../players/O/Oleg_Markov.html
../players/M/Mitch_Grigg.html
../players/S/Sam_Switkowski.html
../players/R/Russell_Johnston.html
../players/M/Mackenzie_Willis.html
../players/S/Shane_Mumford.html
../players/M/Michael_Braun.html
../players/H/Herbie_Matthews1.html
../players/I/Ian_Todd.html
../players/G/Glenn_Murphy.html
../players/T/Tim_Barling.html
../players/A/Ashley_Smith.html
../players/M/Mordecai_Bromberg.html
../players/R/Ross_Growcott.html
../players/A/Andrew_Williams.html
../players/L/Levi_Casboult.html
../players/P/Peter_Higgins.html
../players/A/Aaron_Naughton.html
../players/J/Jack_Petruccelle.html
../players/P/Paul_Weston.html
../players/R/Rodney_Maynard.html
../players/S/Sandy_Hyslop.html
../players/D/Darren_OBrien.html
../players/G/Graeme_Gellie.html
../players/R/Robert_Evans.html
../players/D/Daniel_Foster.html
../players/D/Derek_Hall.html
../players

../players/M/Mark_Williams0.html
../players/B/Brett_Bewley.html
../players/S/Sam_Magill.html
../players/A/Alastair_Lord.html
../players/N/Neale_Daniher.html
../players/P/Paul_Morwood.html
../players/G/Greg_Clark.html
../players/P/Phillip_Pinnell.html
../players/R/Ryan_Davis.html
../players/P/Peter_Kenny.html
../players/J/John_Perry.html
../players/T/Tristan_Xerri.html
../players/B/Brendon_Goddard.html
../players/R/Rodney_Ashman.html
../players/S/Stewart_Loewe.html
../players/D/David_Trotter.html
../players/S/Stephen_Copping.html
../players/M/Matthew_ODwyer.html
../players/S/Saverio_Rocca.html
../players/J/Jack_Silvagni.html
../players/M/Mark_Wall.html
../players/G/Gary_Hardeman.html
../players/N/Noel_Leary.html
../players/M/Michael_Lockman.html
../players/M/Michael_Gale.html
../players/S/Stephen_Edgar.html
../players/B/Barry_Day.html
../players/A/Andrew_McDougall.html
../players/J/Joel_Wilkinson.html
../players/B/Bob_Keddie.html
../players/M/Mitch_Morton.html
../players/M/Mark_Attard.h

In [168]:
len(players_data_list)

5475

## Creating player data

In [172]:
player_df = pd.DataFrame(columns=['name', 'afl_tables_url', 'team', 'year', 'game_num', 'opponent', 'Rd', 'R', 
                                  '#', 'KI', 'MK', 'HB', 'DI', 'GL', 'BH', 'HO', 'TK', 'RB', 'IF', 'CL', 
                                  'CG', 'FF', 'FA', 'BR', 'CP', 'UP', 'CM', 'MI', '1%', 'BO', 'GA', '%P'])
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P


In [173]:
def func(i, data):
    player_url, player_table_list = data[0], data[1]
    print(i, player_url)
    player_name = player_url.split('/')[-1][:-5].replace('_', ' ')
    year_set = {int(season.columns[0][0].split('-')[1].strip()) > 2011 for season in player_table_list}
    print(year_set)
    if not all(year_set):
        return
    for season in player_table_list:
        labels = [d.strip() for d in season.columns[0][0].split('-')]
        if len(labels) != 2:
            raise Exception(f'Error with {player_url}, {season.columns[0]}')
        team_name, year = labels

        for i, row in season.iterrows():
            created_row = [player_name, player_url, team_name, year] + list(row.values)
            player_df.loc[len(player_df)] = created_row

In [174]:
for i, data in enumerate(players_data_list):
    func(i, data)

0 ../players/R/Ron_Andrews.html
{False}
1 ../players/T/Ted_Whitten0.html
{False}
2 ../players/P/Paul_Constance.html
{False}
3 ../players/J/James_Madden.html
{True}
4 ../players/P/Peter_Cloke.html
{False}
5 ../players/J/John_Letcher.html
{False}
6 ../players/J/Joshua_Mail.html
{False}
7 ../players/J/Jarrod_Lienert.html
{True}
8 ../players/C/Cameron_Hunter.html
{False}
9 ../players/M/Michael_Browne.html
{False}
10 ../players/N/Nathan_Lovett-Murray.html
{False, True}
11 ../players/G/Graham_Hunnibell.html
{False}
12 ../players/S/Shane_Kersten.html
{True}
13 ../players/A/Aaron_Lord.html
{False}
14 ../players/J/John_Durnan.html
{False}
15 ../players/M/Mitch_Hallahan.html
{True}
16 ../players/J/Jarryn_Geary.html
{False, True}
17 ../players/R/Ray_Boyanich.html
{False}
18 ../players/B/Brenton_Phillips.html
{False}
19 ../players/B/Beau_Maister.html
{False, True}
20 ../players/J/John_Peter-Budge.html
{False}
21 ../players/B/Ben_Ingleton.html
{False}
22 ../players/I/Ian_Callinan.html
{False, True}

191 ../players/B/Bailey_Banfield.html
{True}
192 ../players/R/Ryley_Stoddart.html
{True}
193 ../players/K/Ken_Timms.html
{False}
194 ../players/J/Joel_Reynolds.html
{False}
195 ../players/F/Fabian_Deluca.html
{False}
196 ../players/E/Eric_Clarke.html
{False}
197 ../players/H/Howard_Tarpey.html
{False}
198 ../players/J/Jackson_Paine.html
{True}
199 ../players/O/Oscar_McDonald.html
{True}
200 ../players/D/Dean_Solomon.html
{False}
201 ../players/J/Justin_Westhoff.html
{False, True}
202 ../players/P/Paul_Ross.html
{False}
203 ../players/B/Brent_Hutton.html
{False}
204 ../players/J/Jai_Serong.html
{True}
205 ../players/R/Russell_Dickson0.html
{False}
206 ../players/P/Paul_Chapman.html
{False, True}
207 ../players/S/Stephen_Anderson.html
{False}
208 ../players/R/Ray_Biffin.html
{False}
209 ../players/P/Peter_German.html
{False}
210 ../players/B/Blaine_Boekhorst.html
{True}
211 ../players/H/Hugh_Greenwood.html
{True}
212 ../players/G/Glenn_Archer.html
{False}
213 ../players/C/Chris_Wittman.h

383 ../players/N/Nathan_Kreuger.html
{True}
384 ../players/K/Kerry_Haywood.html
{False}
385 ../players/A/Allan_Sidebottom.html
{False}
386 ../players/T/Travis_Boak.html
{False, True}
387 ../players/A/Albie_Dunn.html
{False}
388 ../players/J/Josh_Glenn.html
{True}
389 ../players/G/Gary_Odgers.html
{False}
390 ../players/H/Hayden_Crozier.html
{True}
391 ../players/C/Cameron_Wight.html
{False}
392 ../players/C/Charlie_Curnow.html
{True}
393 ../players/L/Louis_Butler.html
{True}
394 ../players/B/Bill_Ryan0.html
{False}
395 ../players/J/Jim_Sewell.html
{False}
396 ../players/G/Geoff_Miles.html
{False}
397 ../players/W/Will_Hayward.html
{True}
398 ../players/P/Peter_Chilton.html
{False}
399 ../players/T/Tadhg_Kennelly.html
{False}
400 ../players/C/Cameron_Doyle.html
{False}
401 ../players/M/Marcus_Ashcroft.html
{False}
402 ../players/M/Matthew_Nicks.html
{False}
403 ../players/J/Jack_Riewoldt.html
{False, True}
404 ../players/G/Garry_Davidson.html
{False}
405 ../players/D/Doug_Cox.html
{Fals

572 ../players/S/Stephen_Jolley.html
{False}
573 ../players/P/Paddy_Ryder.html
{False, True}
574 ../players/G/Greg_Tivendale.html
{False}
575 ../players/M/Matt_McDonough.html
{True}
576 ../players/M/Matthew_Lappin.html
{False}
577 ../players/D/David_Round.html
{False}
578 ../players/N/Nick_Holland.html
{False}
579 ../players/D/Daniel_Connors.html
{False, True}
580 ../players/J/Jake_Edwards.html
{False}
581 ../players/B/Bert_Chapman1.html
{False}
582 ../players/J/Josh_Fraser.html
{False, True}
583 ../players/C/Craig_Balme.html
{False}
584 ../players/M/Mitch_White.html
{True}
585 ../players/D/Damen_Shaw.html
{False}
586 ../players/D/Darren_Bewick.html
{False}
587 ../players/L/Lewis_Roberts-Thomson.html
{False, True}
588 ../players/P/Paul_Licuria.html
{False}
589 ../players/J/Jade_Rawlings.html
{False}
590 ../players/N/Nick_Robertson.html
{True}
591 ../players/D/Doug_Koop.html
{False}
592 ../players/E/Eddie_Lake.html
{False}
593 ../players/T/Tom_Quinn.html
{False}
594 ../players/R/Rory_Hi

760 ../players/J/Joel_Bowden.html
{False}
761 ../players/J/John_Barnes.html
{False}
762 ../players/T/Tom_Campbell1.html
{True}
763 ../players/D/Dustin_Fletcher.html
{False, True}
764 ../players/N/Nick_Ries.html
{False}
765 ../players/J/Jim_Krakouer.html
{False}
766 ../players/G/Graeme_Dunstan.html
{False}
767 ../players/B/Brian_Wilson0.html
{False}
768 ../players/J/Jackson_Ramsay.html
{True}
769 ../players/S/Stephen_Clifford.html
{False}
770 ../players/R/Richard_Radziminski.html
{False}
771 ../players/D/Donald_Dickie.html
{False}
772 ../players/M/Mike_Butcher.html
{False}
773 ../players/A/Aldo_Dipetta.html
{False}
774 ../players/C/Callum_Brown0.html
{True}
775 ../players/B/Brian_Gray.html
{False}
776 ../players/W/Willem_Drew.html
{True}
777 ../players/R/Ross_Smith0.html
{False}
778 ../players/R/Ross_Funcke.html
{False}
779 ../players/J/James_McLure.html
{False}
780 ../players/R/Robert_Handley.html
{False}
781 ../players/P/Phillip_Read.html
{False}
782 ../players/D/David_Fanning.html
{F

...

4538 ../players/G/Greg_Lambert.html
{False}
4539 ../players/C/Chris_Daniher.html
{False}
4540 ../players/D/Dale_Holmes.html
{False}
4541 ../players/L/Luke_Jericho.html
{False}
4542 ../players/N/Neil_MacLeod.html
{False}
4543 ../players/S/Sean_Bowden.html
{False}
4544 ../players/C/Caleb_Graham.html
{True}
4545 ../players/R/Ross_Young.html
{False}
4546 ../players/D/Dean_Ross.html
{False}
4547 ../players/W/Wally_Matera.html
{False}
4548 ../players/M/Michael_Barlow.html
{False, True}
4549 ../players/T/Tanner_Bruhn.html
{True}
4550 ../players/C/Connor_Idun.html
{True}
4551 ../players/L/Leroy_Jetta.html
{False, True}
4552 ../players/R/Renato_Dintinosante.html
{False}
4553 ../players/R/Rhyce_Shaw.html
{False, True}
4554 ../players/B/Ben_Walton.html
{False}
4555 ../players/S/Scott_Freeborn.html
{False}
4556 ../players/J/Jake_Lever.html
{True}
4557 ../players/M/Matt_Shaw.html
{False, True}
4558 ../players/P/Peter_Kiel.html
{False}
4559 ../players/M/Mathew_Capuano.html
{False}
4560 ../players/C/

4721 ../players/E/Eric_Leech.html
{False}
4722 ../players/A/Allen_Daniels.html
{False}
4723 ../players/J/Jason_Akermanis.html
{False}
4724 ../players/A/Alik_Magin.html
{False, True}
4725 ../players/G/Greg_Parke.html
{False}
4726 ../players/B/Bob_Murray.html
{False}
4727 ../players/D/Danny_Dickfos.html
{False}
4728 ../players/C/Clancee_Pearce.html
{False, True}
4729 ../players/I/Ian_Rickman.html
{False}
4730 ../players/N/Nathan_Djerrkura.html
{False, True}
4731 ../players/B/Brendon_Gale.html
{False}
4732 ../players/K/Kris_Barlow.html
{False}
4733 ../players/J/Jack_Payne.html
{True}
4734 ../players/J/Jordon_Sweet.html
{True}
4735 ../players/F/Frank_Gumbleton.html
{False}
4736 ../players/J/Josh_Schoenfeld.html
{True}
4737 ../players/P/Paul_Kelly.html
{False}
4738 ../players/L/Lewis_Melican.html
{True}
4739 ../players/M/Mark_Amos.html
{False}
4740 ../players/P/Peter_Faulks.html
{False, True}
4741 ../players/P/Peter_Burgoyne.html
{False}
4742 ../players/Z/Zach_Tuohy.html
{False, True}
4743 

4908 ../players/M/Matthew_Westhoff.html
{False}
4909 ../players/J/James_Gowans.html
{False}
4910 ../players/I/Israel_Folau.html
{True}
4911 ../players/D/Des_Tuddenham.html
{False}
4912 ../players/S/Stephen_Wrigley.html
{True}
4913 ../players/I/Ian_Cooper0.html
{False}
4914 ../players/J/Josh_Sinn.html
{True}
4915 ../players/M/Matthew_Young.html
{False}
4916 ../players/B/Brett_Knowles.html
{False}
4917 ../players/J/John_Sudholz.html
{False}
4918 ../players/A/Aiden_Begg.html
{True}
4919 ../players/D/Darryl_Herrod.html
{False}
4920 ../players/P/Paul_Williams.html
{False}
4921 ../players/T/Ted_Whitten1.html
{False}
4922 ../players/N/Nick_Wilton.html
{False}
4923 ../players/T/Tony_Liberatore.html
{False}
4924 ../players/K/Kevin_Stevens.html
{False}
4925 ../players/D/David_Schwarz.html
{False}
4926 ../players/B/Brad_Fuller.html
{False}
4927 ../players/L/Luke_Cleary.html
{True}
4928 ../players/C/Corey_Wagner.html
{True}
4929 ../players/C/Changkuoth_Jiath.html
{True}
4930 ../players/J/James_Cou

5102 ../players/L/Lewis_Johnston.html
{False, True}
5103 ../players/T/Tommy_Floyd.html
{False}
5104 ../players/K/Kevin_Dyson.html
{False}
5105 ../players/L/Luke_Power.html
{False, True}
5106 ../players/M/Michael_Johnson.html
{False, True}
5107 ../players/K/Ken_Roberts0.html
{False}
5108 ../players/S/Sam_Day.html
{False, True}
5109 ../players/C/Chris_Newman.html
{False, True}
5110 ../players/T/Tony_Woods.html
{False}
5111 ../players/D/Don_Roach.html
{False}
5112 ../players/W/Will_Sierakowski.html
{True}
5113 ../players/M/Mark_Kellett.html
{False}
5114 ../players/S/Shane_Tuck.html
{False, True}
5115 ../players/R/Ron_Barassi0.html
{False}
5116 ../players/D/Deven_Robertson.html
{True}
5117 ../players/J/Jarrod_Harbrow.html
{False, True}
5118 ../players/D/Damien_Hardwick.html
{False}
5119 ../players/S/Sean_King.html
{False}
5120 ../players/M/Matthew_Kreuzer.html
{False, True}
5121 ../players/T/Tom_Fullarton.html
{True}
5122 ../players/A/Adrian_McAdam.html
{False}
5123 ../players/J/John_Fishe

5285 ../players/L/Lachie_Hunter.html
{True}
5286 ../players/H/Harry_Jones2.html
{True}
5287 ../players/S/Steven_Salopek.html
{False}
5288 ../players/C/Chris_Elliott.html
{False}
5289 ../players/L/Lance_Styles.html
{False}
5290 ../players/J/Jarryd_Morton.html
{False}
5291 ../players/L/Laurie_Dwyer.html
{False}
5292 ../players/M/Mitch_Robinson.html
{False, True}
5293 ../players/T/Tom_McNamara1.html
{False}
5294 ../players/G/Grant_Williams.html
{False}
5295 ../players/G/Graeme_Hatcher.html
{False}
5296 ../players/C/Chris_Johnson1.html
{False}
5297 ../players/D/David_Hynes.html
{False}
5298 ../players/M/Matt_Rosa.html
{False, True}
5299 ../players/D/Doug_Hawkins.html
{False}
5300 ../players/S/Shane_Woewodin.html
{False}
5301 ../players/J/Jack_Lucas.html
{False}
5302 ../players/E/Ed_Curnow.html
{False, True}
5303 ../players/M/Mark_Slater.html
{False}
5304 ../players/J/Josh_Head.html
{False}
5305 ../players/D/David_OConnell.html
{False}
5306 ../players/M/Mike_Pokrovsky.html
{False}
5307 ../p

In [576]:
player_df[['KI','MK','HB','DI','GL','BH','HO','TK','RB','IF','CL','CG','FF','FA','BR','CP', 
           'UP','CM','MI','1%','BO','GA','%P']] = player_df[['KI','MK','HB','DI','GL','BH',
                                                             'HO','TK','RB','IF','CL','CG',
                                                             'FF','FA','BR','CP','UP','CM',
                                                             'MI','1%','BO','GA','%P']].fillna(0)

### Separating off aggregate data from each season into different dataset

In [175]:
player_agg_df = player_df[player_df['opponent'] == 'Totals']

In [176]:
player_df = player_df.drop(player_agg_df.index)

In [177]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52636 entries, 0 to 57298
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            52636 non-null  object 
 1   afl_tables_url  52636 non-null  object 
 2   team            52636 non-null  object 
 3   year            52636 non-null  object 
 4   game_num        52636 non-null  object 
 5   opponent        52636 non-null  object 
 6   Rd              52636 non-null  object 
 7   R               52636 non-null  object 
 8   #               52636 non-null  object 
 9   KI              51979 non-null  float64
 10  MK              49489 non-null  float64
 11  HB              51281 non-null  float64
 12  DI              52202 non-null  float64
 13  GL              17002 non-null  float64
 14  BH              14573 non-null  float64
 15  HO              5730 non-null   float64
 16  TK              45721 non-null  float64
 17  RB              32926 non-null 

## Creating a player bio table

In [202]:
players_bio_list = []
for i, player_url in enumerate(player_df['afl_tables_url'].unique()):
    print(i, player_url)
    page = requests.get('https://afltables.com/afl/stats' + player_url[2:])
    soup = BeautifulSoup(page.text)
    name = soup.find("h1").text
    height_and_weight = soup.text.split('\n')[8]
    birthday_debut = soup.text.split('\n')[7]
    print('name', name, 'h and w', height_and_weight)
    players_bio_list.append((player_url, name, height_and_weight,birthday_debut))

0 ../players/J/James_Madden.html
name James Madden h and w Height:186 cm Weight:85 kg
1 ../players/J/Jarrod_Lienert.html
name Jarrod Lienert h and w Height:195 cm Weight:88 kg
2 ../players/S/Shane_Kersten.html
name Shane Kersten h and w Height:191 cm Weight:90 kg
3 ../players/M/Mitch_Hallahan.html
name Mitch Hallahan h and w Height:183 cm Weight:84 kg
4 ../players/J/Jack_Lonie.html
name Jack Lonie h and w Height:175 cm Weight:70 kg
5 ../players/I/Irving_Mosquito.html
name Irving Mosquito h and w Height:176 cm Weight:72 kg
6 ../players/A/Andrew_McInnes.html
name Andrew McInnes h and w Height:191 cm Weight:87 kg
7 ../players/S/Sam_Berry.html
name Sam Berry h and w Height:181 cm Weight:81 kg
8 ../players/B/Brad_McKenzie.html
name Brad McKenzie h and w Height:186 cm Weight:84 kg
9 ../players/L/Logan_Austin.html
name Logan Austin h and w Height:195 cm Weight:85 kg
10 ../players/N/Nathan_Freeman.html
name Nathan Freeman h and w Height:183 cm Weight:88 kg
11 ../players/K/Kurt_Mutimer.html
nam

name Brayden Fiorini h and w Height:186 cm Weight:82 kg
94 ../players/P/Patrick_Naish.html
name Patrick Naish h and w Height:182 cm Weight:75 kg
95 ../players/C/Conor_McKenna.html
name Conor McKenna h and w Height:184 cm Weight:88 kg
96 ../players/K/Kyle_Langford.html
name Kyle Langford h and w Height:191 cm Weight:87 kg
97 ../players/B/Bradley_Hill.html
name Bradley Hill h and w Height:179 cm Weight:79 kg
98 ../players/M/Matt_Guelfi.html
name Matt Guelfi h and w Height:184 cm Weight:85 kg
99 ../players/L/Luke_Dunstan.html
name Luke Dunstan h and w Height:185 cm Weight:84 kg
100 ../players/J/Josh_Clayton.html
name Josh Clayton h and w Height:190 cm Weight:80 kg
101 ../players/K/Kade_Kolodjashnij.html
name Kade Kolodjashnij h and w Height:190 cm Weight:85 kg
102 ../players/J/Jamarra_Ugle-Hagan.html
name Jamarra Ugle-Hagan h and w Height:194 cm Weight:84 kg
103 ../players/A/Adam_Carter.html
name Adam Carter h and w Height:179 cm Weight:74 kg
104 ../players/S/Sam_Grimley.html
name Sam Gri

name Ben Brown h and w Height:200 cm Weight:101 kg
186 ../players/C/Charlie_Comben.html
name Charlie Comben h and w Height:199 cm Weight:82 kg
187 ../players/C/Conor_Stone.html
name Conor Stone h and w Height:188 cm Weight:81 kg
188 ../players/L/Lachie_Ash.html
name Lachie Ash h and w Height:186 cm Weight:80 kg
189 ../players/D/Dylan_Williams.html
name Dylan Williams h and w Height:185 cm Weight:79 kg
190 ../players/M/Max_Duffy.html
name Max Duffy h and w Height:184 cm Weight:77 kg
191 ../players/J/Jacob_Hopper.html
name Jacob Hopper h and w Height:187 cm Weight:87 kg
192 ../players/A/Adam_Saad.html
name Adam Saad h and w Height:178 cm Weight:78 kg
193 ../players/G/Griffin_Logue.html
name Griffin Logue h and w Height:193 cm Weight:101 kg
194 ../players/S/Sam_Dunell.html
name Sam Dunell h and w Height:190 cm Weight:81 kg
195 ../players/S/Sam_Menegola.html
name Sam Menegola h and w Height:189 cm Weight:89 kg
196 ../players/F/Fergus_Greene.html
name Fergus Greene h and w Height:187 cm Wei

name Jordan Dawson h and w Height:192 cm Weight:86 kg
279 ../players/K/Kayne_Turner.html
name Kayne Turner h and w Height:180 cm Weight:75 kg
280 ../players/C/Colin_ORiordan.html
name Colin ORiordan h and w Height:187 cm Weight:87 kg
281 ../players/J/Jackson_Callow.html
name Jackson Callow h and w Height:194 cm Weight:95 kg
282 ../players/M/Marc_Pittonet.html
name Marc Pittonet h and w Height:202 cm Weight:105 kg
283 ../players/M/Mason_Wood.html
name Mason Wood h and w Height:192 cm Weight:87 kg
284 ../players/J/Jack_Saunders.html
name Jack Saunders h and w Height:180 cm Weight:76 kg
285 ../players/C/Connor_Blakely.html
name Connor Blakely h and w Height:188 cm Weight:90 kg
286 ../players/N/Noah_Anderson.html
name Noah Anderson h and w Height:188 cm Weight:86 kg
287 ../players/J/Jackson_Hately.html
name Jackson Hately h and w Height:190 cm Weight:83 kg
288 ../players/K/Kristian_Jaksch.html
name Kristian Jaksch h and w Height:196 cm Weight:90 kg
289 ../players/I/Ivan_Soldo.html
name Iva

name Alex Pearce h and w Height:201 cm Weight:101 kg
372 ../players/J/Jye_Menzie.html
name Jye Menzie h and w Height:180 cm Weight:81 kg
373 ../players/K/Kurt_Heatherley.html
name Kurt Heatherley h and w Height:193 cm Weight:89 kg
374 ../players/T/Tobe_Watson.html
name Tobe Watson h and w Height:190 cm Weight:83 kg
375 ../players/B/Bailey_Scott.html
name Bailey Scott h and w Height:186 cm Weight:73 kg
376 ../players/C/Charlie_Ballard.html
name Charlie Ballard h and w Height:195 cm Weight:91 kg
377 ../players/M/Max_Lynch.html
name Max Lynch h and w Height:200 cm Weight:99 kg
378 ../players/A/Andrew_Brayshaw.html
name Andrew Brayshaw h and w Height:184 cm Weight:84 kg
379 ../players/H/Hugh_Goddard.html
name Hugh Goddard h and w Height:196 cm Weight:94 kg
380 ../players/D/Declan_Keilty.html
name Declan Keilty h and w Height:194 cm Weight:94 kg
381 ../players/C/Caolan_Mooney.html
name Caolan Mooney h and w Height:184 cm Weight:76 kg
382 ../players/B/Ben_McNiece.html
name Ben McNiece h and 

...

name Tim Kelly h and w Height:182 cm Weight:81 kg
652 ../players/B/Blayne_Wilson.html
name Blayne Wilson h and w Height:188 cm Weight:89 kg
653 ../players/R/Riley_Collier-Dawkins.html
name Riley Collier-Dawkins h and w Height:189 cm Weight:89 kg
654 ../players/J/Jonathan_Freeman.html
name Jonathan Freeman h and w Height:198 cm Weight:96 kg
655 ../players/D/Denver_Grainger-Barras.html
name Denver Grainger-Barras h and w Height:195 cm Weight:78 kg
656 ../players/Z/Zak_Jones.html
name Zak Jones h and w Height:181 cm Weight:80 kg
657 ../players/R/Reef_McInnes.html
name Reef McInnes h and w Height:194 cm Weight:87 kg
658 ../players/R/Reilly_OBrien.html
name Reilly OBrien h and w Height:202 cm Weight:101 kg
659 ../players/J/Jason_Ashby.html
name Jason Ashby h and w Height:187 cm Weight:77 kg
660 ../players/B/Brant_Colledge.html
name Brant Colledge h and w Height:192 cm Weight:90 kg
661 ../players/L/Louis_Herbert.html
name Louis Herbert h and w Height:188 cm Weight:77 kg
662 ../players/J/Joe_

name Shane McAdam h and w Height:181 cm Weight:78 kg
745 ../players/Z/Zac_Williams.html
name Zac Williams h and w Height:185 cm Weight:84 kg
746 ../players/S/Scott_Jones.html
name Scott Jones h and w Height:203 cm Weight:115 kg
747 ../players/C/Cameron_Polson.html
name Cameron Polson h and w Height:177 cm Weight:76 kg
748 ../players/L/Lachlan_Murphy.html
name Lachlan Murphy h and w Height:176 cm Weight:76 kg
749 ../players/R/Rhett_Bazzo.html
name Rhett Bazzo h and w Height:195 cm Weight:86 kg
750 ../players/J/Josh_Dunkley.html
name Josh Dunkley h and w Height:190 cm Weight:88 kg
751 ../players/M/Mason_Redman.html
name Mason Redman h and w Height:187 cm Weight:87 kg
752 ../players/J/John_Noble.html
name John Noble h and w Height:180 cm Weight:72 kg
753 ../players/J/Jesse_Joyce.html
name Jesse Joyce h and w Height:191 cm Weight:84 kg
754 ../players/N/Nathan_Murphy.html
name Nathan Murphy h and w Height:190 cm Weight:79 kg
755 ../players/J/James_Aish.html
name James Aish h and w Height:18

name Nakia Cockatoo h and w Height:186 cm Weight:90 kg
838 ../players/S/Sam_Darley.html
name Sam Darley h and w Height:186 cm Weight:82 kg
839 ../players/J/Jesse_Motlop.html
name Jesse Motlop h and w Height:176 cm Weight:76 kg
840 ../players/D/Daniel_Venables.html
name Daniel Venables h and w Height:187 cm Weight:84 kg
841 ../players/B/Braeden_Campbell.html
name Braeden Campbell h and w Height:181 cm Weight:73 kg
842 ../players/D/Darcy_Wilmot.html
name Darcy Wilmot h and w Height:183 cm Weight:75 kg
843 ../players/D/Darcy_Macpherson.html
name Darcy Macpherson h and w Height:174 cm Weight:78 kg
844 ../players/F/Flynn_Appleby.html
name Flynn Appleby h and w Height:187 cm Weight:82 kg
845 ../players/T/Tom_Curren.html
name Tom Curren h and w Height:178 cm Weight:78 kg
846 ../players/B/Billy_Frampton.html
name Billy Frampton h and w Height:200 cm Weight:90 kg
847 ../players/S/Shane_Yarran.html
name Shane Yarran h and w Height:185 cm Weight:79 kg
848 ../players/A/Alec_Waterman.html
name Alec

name Luke Davies-Uniacke h and w Height:188 cm Weight:85 kg
932 ../players/D/Darcy_Byrne-Jones.html
name Darcy Byrne-Jones h and w Height:181 cm Weight:75 kg
933 ../players/T/Tom_De_Koning.html
name Tom De Koning h and w Height:203 cm Weight:97 kg
934 ../players/H/Hayden_Young.html
name Hayden Young h and w Height:188 cm Weight:82 kg
935 ../players/S/Sam_Mayes.html
name Sam Mayes h and w Height:189 cm Weight:88 kg
936 ../players/T/Troy_Menzel.html
name Troy Menzel h and w Height:187 cm Weight:84 kg
937 ../players/J/Jordan_Clark.html
name Jordan Clark h and w Height:185 cm Weight:78 kg
938 ../players/D/Dom_Sheed.html
name Dom Sheed h and w Height:185 cm Weight:87 kg
939 ../players/B/Brent_Daniels.html
name Brent Daniels h and w Height:170 cm Weight:74 kg
940 ../players/O/Oleg_Markov.html
name Oleg Markov h and w Height:188 cm Weight:82 kg
941 ../players/M/Mitch_Grigg.html
name Mitch Grigg h and w Height:182 cm Weight:81 kg
942 ../players/S/Sam_Switkowski.html
name Sam Switkowski h and w

name Jed Anderson h and w Height:179 cm Weight:81 kg
1024 ../players/W/Will_Walker.html
name Will Walker h and w Height:186 cm Weight:78 kg
1025 ../players/R/Reuben_Ginbey.html
name Reuben Ginbey h and w Height:191 cm Weight:83 kg
1026 ../players/M/Mitch_McGovern.html
name Mitch McGovern h and w Height:191 cm Weight:93 kg
1027 ../players/J/Jack_Macrae.html
name Jack Macrae h and w Height:191 cm Weight:85 kg


In [235]:
player_table_data = []
for player in players_bio_list:
    try:
        url = player[0]
        name = player[1]
        height = re.split(' |:',player[2])[1]
        weight = re.split(' |:',player[2])[4]
        dob = re.split(':| |\(',player[3])[9]
        debut_list = [re.split(':| |\(',player[3])[12]]
        if re.split(':| |\(',player[3])[13] != 'Last':
            debut_list.append(re.split(':| |\(',player[3])[13])
            last_list = [re.split(':| |\(',player[3])[15]]
            try:
                last_list.append(re.split(':| |\(',player[3])[16])
            except:
                pass
        else:
            last_list = [re.split(':| |\(',player[3])[14]]
            try:
                last_list.append(re.split(':| |\(',player[3])[15])
            except:
                pass
        last = ' '.join(last_list)
        debut = ' '.join(debut_list)
        player_table_data.append((url, name, height, weight, dob, debut, last))
    except:
        print(player)
        break

In [238]:
player_data_df = pd.DataFrame(player_table_data, columns = ['afl_tables_url', 'name', 'height', 
                                                            'weight', 'dob', 'debut', 'last'])

In [246]:
player_data_df['height'] = player_data_df['height'].astype(int)
player_data_df['weight'] = player_data_df['weight'].astype(int)
player_data_df['dob'] = pd.to_datetime(player_data_df['dob'], format="%d-%b-%Y")

In [248]:
player_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   afl_tables_url  1028 non-null   object        
 1   name            1028 non-null   object        
 2   height          1028 non-null   int32         
 3   weight          1028 non-null   int32         
 4   dob             1028 non-null   datetime64[ns]
 5   debut           1028 non-null   object        
 6   last            1028 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(4)
memory usage: 48.3+ KB


In [298]:
player_data_df['debut_year'] = player_data_df['debut'].apply(lambda x: 0 \
        if len(re.findall(r'\d+(?=y)', x)) == 0 else re.findall(r'\d+(?=y)', x)[0]).astype(int)
player_data_df['debut_day'] = player_data_df['debut'].apply(lambda x: 0 \
        if len(re.findall(r'y (\d+)d', x)) == 0 else re.findall(r'y (\d+)d', x)[0].strip()).astype(int)

player_data_df['last_year'] = player_data_df['last'].apply(lambda x: 0 \
        if len(re.findall(r'\d+(?=y)', x)) == 0 else re.findall(r'\d+(?=y)', x)[0]).astype(int)
player_data_df['last_day'] = player_data_df['last'].apply(lambda x: 0 \
        if len(re.findall(r'y (\d+)d', x)) == 0 else re.findall(r'y (\d+)d', x)[0].strip()).astype(int)

In [310]:
today = datetime.today()
def get_date(row, column):
    return (today - relativedelta(years=row[f'{column}_year'], days=row[f'{column}_day'])).date()

In [311]:
player_data_df['debut_date'] = player_data_df.apply(lambda x: get_date(x, 'debut'), axis=1)
player_data_df['last_date'] = player_data_df.apply(lambda x: get_date(x, 'last'), axis=1)

In [317]:
player_data_df = player_data_df.drop(['debut', 'last', 'debut1', 'debut_year', 'debut_day',
                                      'last_year', 'last_day'], axis=1)

## Creating a games stats table

In [471]:
game_df = pd.DataFrame(columns=['year','round','date_time', 'venue', 'home_team', 'away_team', 
                                'attendance', 'home_team_quarter_score', 'away_team_quarter_score', 
                                'home_team_final_score', 'away_team_final_score'])
game_df

,year,round,date_time,venue,home_team,away_team,attendance,home_team_quarter_score,away_team_quarter_score,home_team_final_score,away_team_final_score


In [472]:
round_list = ['Round 1','Round 2','Round 3','Round 4','Round 5','Round 6','Round 7','Round 8',
              'Round 9','Round 10','Round 11','Round 12','Round 13','Round 14','Round 15','Round 16',
              'Round 17','Round 18','Round 19','Round 20','Round 21','Round 22','Round 23',
              'Qualifying Final','Elimination Final','Semi Final','Preliminary Final','Grand Final']

year_list = sorted(list(player_df['year'].unique()))

In [473]:
for year in year_list:
    game_tables = pd.read_html(f'https://afltables.com/afl/seas/{year}.html')
    for game_table in game_tables:
        try:
            if len(game_table) == 1:
                if game_table[0][0] in round_list:
                    rnd_num = game_table[0][0]
                else:
                    continue
            elif len(game_table) == 2:
                home_team = game_table[0][0]
                away_team = game_table[0][1]
                home_team_quarter_scores = game_table[1][0]
                away_team_quarter_scores = game_table[1][1]
                home_team_final_score = game_table[2][0]
                away_team_final_score = game_table[2][1]
                d = re.split(' Att: | Venue: ',game_table[3][0])
                if len(d) == 3:
                    date_time = d[0]
                    attendance = d[1]
                    venue = d[2]
                elif len(d) == 2:
                    date_time = d[0]
                    attendance = 0
                    venue = d[1]
                else:
                    raise Exception(f'Unexpected data {d}')
                created_row = (year, rnd_num, date_time, venue, home_team, away_team, attendance,
                               home_team_quarter_scores, away_team_quarter_scores, 
                               home_team_final_score, away_team_final_score)
                game_df.loc[len(game_df)] = created_row
            else:
                continue
        except Exception as e:
            print(f'{year}, {game_table}')
            raise Exception(e)

In [474]:
game_df = game_df[~game_df['home_team_final_score'].isna()]

In [477]:
game_df[['day_of_week','date','AEST_time',
         'AEST_am_pm','local_time','local_am_pm']] = game_df['date_time'].str.split(' ',expand=True)
game_df = game_df.drop('date_time', axis=1)

C:\Users\aaron\AppData\Local\Temp\ipykernel_19884\1222839064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_df[['day_of_week','date','AEST_time',


In [479]:
game_df['date'] = pd.to_datetime(game_df['date'], format="%d-%b-%Y")
game_df['local_time'] = game_df['local_time'].apply(lambda x: x[1:] if x else x)
game_df['local_am_pm'] = game_df['local_am_pm'].apply(lambda x: x[:-1] if x else x)

In [480]:
game_df['AEST'] = pd.to_datetime(game_df['AEST_time']+' '+game_df['AEST_am_pm'], format="%I:%M %p").dt.time
game_df['local'] = pd.to_datetime(game_df['local_time']+' '+game_df['local_am_pm'], format="%I:%M %p").dt.time

In [483]:
game_df = game_df.drop(['AEST_time', 'AEST_am_pm', 'local_time', 'local_am_pm'], axis=1)

In [489]:
game_df['local'] = game_df['local'].fillna(game_df['AEST'])

In [499]:
game_df['home_team_quarter_score'].str.split(' ')

0            [1.2, 3.3, 3.4, 5.7]
1          [3.4, 5.6, 10.7, 12.9]
2       [2.1, 10.6, 14.10, 20.17]
3          [3.3, 7.4, 8.8, 11.12]
4           [3.1, 5.3, 8.6, 10.8]
                  ...            
2253        [5.4, 7.5, 9.7, 14.8]
2254       [6.1, 9.3, 11.5, 13.8]
2255      [2.3, 3.9, 6.11, 10.13]
2256       [6.1, 9.3, 14.4, 21.8]
2257      [5.1, 9.5, 10.8, 16.12]
Name: home_team_quarter_score, Length: 2258, dtype: object

In [545]:
game_df[['home_Q1', 'home_Q2', 'home_Q3', 'home_Q4', 'home_extra_time']] = game_df['home_team_quarter_score'].str.split(re.compile(r'[ |\xa0]+'), regex=True, expand=True)
game_df[['away_Q1', 'away_Q2', 'away_Q3', 'away_Q4', 'away_extra_time']] = game_df['away_team_quarter_score'].str.split(re.compile(r'[ |\xa0]+'), regex=True, expand=True)

game_df = game_df.drop(['home_team_quarter_score','away_team_quarter_score'], axis=1)

In [560]:
game_df['round'] = game_df['round'].replace({'Qualifying Final':'QF','Elimination Final':'EF',
                                             'Semi Final':'SF','Preliminary Final':'PF','Grand Final':'GF'})
game_df['round'] = game_df['round'].str.replace('Round ', '')

In [564]:
player_data_df

,afl_tables_url,name,height,weight,dob,debut_date,last_date
0,../players/J/James_Madden.html,James Madden,186,85,1999-11-15,2001-10-25,2000-08-16
1,../players/J/Jarrod_Lienert.html,Jarrod Lienert,195,88,1994-08-05,1999-04-25,1995-03-25
2,../players/S/Shane_Kersten.html,Shane Kersten,191,90,1993-03-15,2002-01-02,1997-10-29
3,../players/M/Mitch_Hallahan.html,Mitch Hallahan,183,84,1992-08-23,2001-08-12,1998-04-07
4,../players/J/Jack_Lonie.html,Jack Lonie,175,70,1996-08-13,2004-08-18,1998-06-11
...,...,...,...,...,...,...,...
1023,../players/J/Jed_Anderson.html,Jed Anderson,179,81,1994-02-02,2004-02-12,1994-09-30
1024,../players/W/Will_Walker.html,Will Walker,186,78,1999-03-30,2003-12-11,2001-11-23
1025,../players/R/Reuben_Ginbey.html,Reuben Ginbey,191,83,2004-09-10,2004-10-03,2004-09-18
1026,../players/M/Mitch_McGovern.html,Mitch McGovern,191,93,1994-10-11,2001-10-25,1994-10-20


In [667]:
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P
0,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,1.0,Port Adelaide,7,W,42,4.0,...,0.0,0.0,10.0,2.0,0.0,0.0,3.0,0.0,0.0,78.0
1,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,2.0,Richmond,10,W,42,7.0,...,1.0,0.0,9.0,3.0,2.0,0.0,0.0,0.0,0.0,77.0
2,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,3.0,Greater Western Sydney,11,W,42,11.0,...,0.0,0.0,5.0,9.0,0.0,0.0,3.0,0.0,0.0,82.0
3,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,4.0,North Melbourne,14,W,42,9.0,...,0.0,0.0,6.0,7.0,0.0,0.0,6.0,0.0,0.0,74.0
4,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,5.0,Geelong,15,W,42,2.0,...,0.0,0.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57293,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,207.0,Hawthorn,23,W,11,13.0,...,1.0,0.0,8.0,19.0,0.0,0.0,2.0,0.0,1.0,79.0
57294,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,208.0,Fremantle,EF,L,11,19.0,...,0.0,0.0,13.0,23.0,0.0,0.0,2.0,0.0,0.0,81.0
57296,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,209.0,Melbourne,1,L,11,14.0,...,1.0,0.0,13.0,20.0,0.0,0.0,0.0,0.0,1.0,78.0
57297,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,210.0,St Kilda,2,L,11,7.0,...,0.0,0.0,9.0,18.0,0.0,0.0,3.0,0.0,1.0,78.0


In [565]:
game_df

,year,round,venue,home_team,away_team,attendance,home_team_final_score,away_team_final_score,day_of_week,date,...,home_Q1,home_Q2,home_Q3,home_Q4,home_extra_time,away_Q1,away_Q2,away_Q3,away_Q4,away_extra_time
0,2012,1,Stadium Australia,Greater Western Sydney,Sydney,"38,203",37.0,100.0,Sat,2012-03-24,...,1.2,3.3,3.4,5.7,None,4.1,8.4,13.8,14.16,None
1,2012,1,M.C.G.,Richmond,Carlton,"78,285",81.0,125.0,Thu,2012-03-29,...,3.4,5.6,10.7,12.9,None,3.2,8.7,11.13,18.17,None
2,2012,1,M.C.G.,Hawthorn,Collingwood,"78,466",137.0,115.0,Fri,2012-03-30,...,2.1,10.6,14.10,20.17,None,2.7,7.9,12.16,16.19,None
3,2012,1,M.C.G.,Melbourne,Brisbane Lions,"33,473",78.0,119.0,Sat,2012-03-31,...,3.3,7.4,8.8,11.12,None,1.4,7.8,13.13,17.17,None
4,2012,1,Carrara,Gold Coast,Adelaide,"12,790",68.0,137.0,Sat,2012-03-31,...,3.1,5.3,8.6,10.8,None,7.8,11.10,15.16,19.23,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2253,2023,3,M.C.G.,St Kilda,Essendon,"69,255",92.0,74.0,Sat,2023-04-01,...,5.4,7.5,9.7,14.8,None,1.1,4.5,6.6,11.8,None
2254,2023,3,Adelaide Oval,Port Adelaide,Adelaide,"48,962",86.0,117.0,Sat,2023-04-01,...,6.1,9.3,11.5,13.8,None,5.1,9.3,11.8,18.9,None
2255,2023,3,Carrara,Gold Coast,Geelong,"13,122",73.0,54.0,Sun,2023-04-02,...,2.3,3.9,6.11,10.13,None,3.4,3.6,5.10,7.12,None
2256,2023,3,M.C.G.,Melbourne,Sydney,"42,423",134.0,84.0,Sun,2023-04-02,...,6.1,9.3,14.4,21.8,None,1.3,6.6,9.9,12.12,None


## Creating player position and draft data

In [582]:
combined_df=player_df.drop('name',axis=1).merge(player_data_df, on='afl_tables_url')

In [595]:
combined_df['team'].unique()

array(['Brisbane Lions', 'Port Adelaide', 'St Kilda', 'Geelong',
       'Fremantle', 'Hawthorn', 'Gold Coast', 'Essendon', 'Carlton',
       'Adelaide', 'North Melbourne', 'West Coast', 'Collingwood',
       'Greater Western Sydney', 'Sydney', 'Melbourne',
       'Western Bulldogs', 'Richmond'], dtype=object)

In [672]:
footywire_map = {'Brisbane Lions':'brisbane-lions', 'Port Adelaide':'port-adelaide-power', 
                 'St Kilda':'st-kilda-saints', 'Geelong':'geelong-cats',
                 'Fremantle':'fremantle-dockers', 'Hawthorn':'hawthorn-hawks', 
                 'Gold Coast':'gold-coast-suns', 'Essendon':'essendon-bombers', 
                 'Carlton':'carlton-blues', 'Adelaide':'adelaide-crows', 
                 'North Melbourne':'kangaroos', 'West Coast':'west-coast-eagles', 
                 'Collingwood':'collingwood-magpies', 'Greater Western Sydney':'greater-western-sydney-giants', 
                 'Sydney':'sydney-swans', 'Melbourne':'melbourne-demons',
                 'Western Bulldogs':'western-bulldogs', 'Richmond':'richmond-tigers'}

footywire_name_map = {'jack':'jackson', 'ollie':'oliver', 'zach':'zachary', 'josh':'joshua', 'lachie':'lachlan', 
                      'dom':'dominic', 'tom':'thomas', 'cam':'cameron', 'nick':'nicholas', 'tim':'timothy',
                      'will':'william', 'mitch':'mitchell', 'matt':'matthew', 'brad':'bradley', 'alex':'alexis',
                      'sam':'samuel', 'timmy':'tim', 'harry':'harrison', 'matthew':'matt', 'brett':'bret', 
                      'zac':'zachary', 'dan':'daniel', 'ed':'edward', 'steve':'stephen', 'bobby':'robert', 
                      'robbie':'robert', 'ned':'edward', 'pat':'patrick', 'nic':'nicholas', 'junior':'willie',
                      'nik':'nicholas'}

url_map = {'pp-fremantle-dockers--michael-frederick':'pp-fremantle-dockers--minairo-frederick',
            'pp-greater-western-sydney-giants--robert-hill':'pp-greater-western-sydney-giants--ian-hill',
            'pp-adelaide-crows--elliott-himmelberg':'pp-adelaide-crows--elliot-himmelberg',
            'pp-greater-western-sydney-giants--zachary-langdon':'pp-west-coast-eagles--zac-giles-langdon',
            'pp-essendon-bombers--nicholas-cox':'pp-essendon-bombers--nikolas-cox',
            'pp-hawthorn-hawks--angus-dewar':'pp-west-coast-eagles--angus-litherland',
            'pp-west-coast-eagles--zachary-langdon':'pp-west-coast-eagles--zac-giles-langdon',
            'pp-essendon-bombers--cory-dellolio':'pp-essendon-bombers--corey-dell-olio',
            'pp-st-kilda-saints--mitchell-o-wens':'pp-st-kilda-saints--mitchito-owens',
            'pp-essendon-bombers--massimo-dambrosio':'pp-essendon-bombers--massimo-d-ambrosio',
            'pp-carlton-blues--joshua-deluca':'pp-carlton-blues--josh-deluca-cardillo',
            'pp-kangaroos--lachlan-hosie':'pp-kangaroos--lachie-hosie',
            'pp-fremantle-dockers--joshua-deluca':'pp-carlton-blues--josh-deluca-cardillo',
            'pp-collingwood-magpies--robert-hill':'pp-collingwood-magpies--ian-hill',
            'pp-fremantle-dockers--jason-carter':'pp-fremantle-dockers--jason-carter-1',
            'pp-geelong-cats--oliver-dempsey':'pp-geelong-cats--ollie-dempsey',
            'pp-west-coast-eagles--angus-dewar':'pp-west-coast-eagles--angus-litherland'}

def get_footywire_url(team, player_name):
    return f'https://www.footywire.com/afl/footy/pp-{footywire_map[team]}--{player_name}'

In [677]:
extra_player_data_list = []
count = 0
for afl_tables_url, name, team in list(combined_df[['afl_tables_url', 'name', 'team']].value_counts().index):
    count += 1
    if count % 100 == 0:
        print(count)
    flag = None
    player_name = name.lower().replace(' ', '-')
    while True:
        url_string = get_footywire_url(team, player_name)
        try:
            page = requests.get(url_string)
        except:
            print(afl_tables_url, name, team, url_string)
        soup = BeautifulSoup(page.text)
        if not soup.find(id="playerProfileData"):
            if not flag:
                player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
                flag = 1
            elif flag == 1:
                flag = 2
                if name.lower().split(' ')[-1][0] != 'o':
                    flag = 4
                    break
                name = ' '.join(name.lower().split(' ')[:-1]) + ' ' + name.lower().split(' ')[-1][0] \
                            + ' ' + name.lower().split(' ')[-1][1:]
                player_name = '-'.join([n for n in name.lower().split(' ')])
            elif flag == 2:
                player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
                flag=3
            else:
                flag = 4
                break
        else:
            break
    if flag == 4:
        player_name = '-'.join([footywire_name_map.get(n,n) for n in name.lower().split(' ')])
        url_string = get_footywire_url(team, player_name)
        url_string = '/'.join(url_string.split('/')[:-1])+'/'+url_map.get(url_string.split('/')[-1], url_string.split('/')[-1])
        try:
            page = requests.get(url_string)
        except:
            print(afl_tables_url, name, team, url_string)
        soup = BeautifulSoup(page.text)
        if not soup.find(id="playerProfileData"):
            print(afl_tables_url, name, team, url_string)
            continue
    try:
        if soup.find(id="playerProfileTeamDiv"):
            team_data = soup.find(id="playerProfileTeamDiv").find('b').text.replace('\xa0', '').replace('\n', '')
        else:
            team_data = None
        if soup.find(id="playerProfileData1"):
            data_1 = soup.find(id="playerProfileData1").text.replace('\xa0', '').replace('\n', ' ')
        else:
            data_1 = None
        if soup.find(id="playerProfileData2"):    
            data_2 = soup.find(id="playerProfileData2").text.replace('\xa0', '').replace('\n', ' ')
        else:
            data_2 = None
        if soup.find(id="playerProfileDraftInfo"):
            draft_data = soup.find(id="playerProfileDraftInfo").text.replace('\xa0', '').replace('\n', ' ')
        else:
            draft_data = None
    except AttributeError:
        print(afl_tables_url, name, team, url_string)
        continue
    extra_player_data_list.append((afl_tables_url, team, team_data, data_1, data_2, draft_data))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [678]:
extra_df = pd.DataFrame(extra_player_data_list, columns=['afl_tables_url', 'team', 'team_data', 
                                                         'data_1', 'data_2', 'draft_data'])

,afl_tables_url,team,team_data,data_1,data_2,draft_data
0,../players/D/Dayne_Zorko.html,Brisbane Lions,#15,"Age: 34yr 2mth Games: 233 Born: February 9, ...",Height: 175cm Weight: 82kg Position: ...,None
1,../players/M/Mark_Blicavs.html,Geelong,#46,"Age: 32yr Games: 231 Born: March 28, 1991 ...",Height: 198cm Weight: 101kg Position: ...,"Last Drafted: Round 3, Pick #54 2011 Rookie D..."
2,../players/D/Dane_Rampe.html,Sydney,#24,"Age: 32yr 10mth Games: 219 Born: June 2, 199...",Height: 188cm Weight: 91kg Position: ...,"Last Drafted: Round 6, Pick #90 2013 National..."
3,../players/J/Jack_Macrae.html,Western Bulldogs,#11,"Age: 28yr 8mth Games: 212 Born: August 3, 19...",Height: 192cm Weight: 88kg Position: ...,"Last Drafted: Round 1, Pick #6 2012 National ..."
4,../players/O/Ollie_Wines.html,Port Adelaide,#16,"Age: 28yr 6mth Games: 207 Born: October 7, 1...",Height: 187cm Weight: 98kg Position: ...,"Last Drafted: Round 1, Pick #7 2012 National ..."
...,...,...,...,...,...,...
1302,../players/J/James_Tsitas.html,Gold Coast,#21,"Age: 28yr 1mth Games: 1 Born: March 3, 1995 ...",Height: 181cm Weight: 84kg Position: ...,Last Drafted: 2022 SSP Recruit
1303,../players/J/Josh_Sinn.html,Port Adelaide,#8,"Age: 20yr 3mth Games: 1 Born: January 7, 200...",Height: 187cm Weight: 78kg Position: ...,"Last Drafted: Round 1, Pick #12 2021 National..."
1304,../players/K/Kayle_Kirby.html,Collingwood,Collingwood Magpies,"Games: 1 Born: October 26, 1998 Origin: Ben...",Playing Height: 182cm Playing Weight: 89kg...,"Last Drafted: Round 3, Pick #50 2016 National..."
1305,../players/K/Keegan_Brooksby.html,Hawthorn,Hawthorn Hawks,"Games: 15 Born: April 27, 1990 Origin: Sout...",Playing Height: 197cm Playing Weight: 105k...,Last Drafted: 2020 SSP Recruit


In [747]:
extra_df[['misc', 'origin']] = extra_df['data_1'].str.split('Origin: ', expand=True)
extra_df[['misc2', 'position']] = extra_df['data_2'].str.split('Position: ', expand=True)
extra_df['draft_round'] = extra_df['draft_data'].str.extract(r'Round\s(\d+)\s*,', expand=False).astype(float)
extra_df['draft_pick_number'] = extra_df['draft_data'].str.extract(r'#(\d+)* ', expand=False).astype(float)
extra_df['draft_year'] = extra_df['draft_data'].str.extract(r'Last Drafted:.*(\d{4})', expand=False)
extra_df['extra_draft_data'] = extra_df['draft_data'].str.extract(r'Last Drafted:.*\d{4}.*?(.*)', expand=True)
extra_df[['draft_type', 'draft_team']] = extra_df['extra_draft_data'].str.split(' by ', expand=True)

extra_df = extra_df.drop(['team_data', 'data_1', 'data_2', 'draft_data', 'misc', 
                          'misc2', 'extra_draft_data'], axis=1)
extra_df

,afl_tables_url,team,origin,position,draft_round,draft_pick_number,draft_year,draft_type,draft_team
0,../players/D/Dayne_Zorko.html,Brisbane Lions,Broadbeach,Forward,NaN,NaN,None,NaN,NaN
1,../players/M/Mark_Blicavs.html,Geelong,Geelong Vfl,"Midfield, Ruck",3.0,54.0,2011,Rookie Draft,Geelong Cats
2,../players/D/Dane_Rampe.html,Sydney,Williamstown,Defender,6.0,90.0,2013,National Draft,Sydney Swans
3,../players/J/Jack_Macrae.html,Western Bulldogs,Oakleigh Chargers,Midfield,1.0,6.0,2012,National Draft,Western Bulldogs
4,../players/O/Ollie_Wines.html,Port Adelaide,Bendigo Pioneers,Midfield,1.0,7.0,2012,National Draft,Port Adelaide Power
...,...,...,...,...,...,...,...,...,...
1302,../players/J/James_Tsitas.html,Gold Coast,Geelong Falcons,"Midfield, Forward ...",NaN,NaN,2022,SSP Recruit,None
1303,../players/J/Josh_Sinn.html,Port Adelaide,Sandringham Dragons,"Midfield, Forward ...",1.0,12.0,2021,National Draft,Port Adelaide Power
1304,../players/K/Kayle_Kirby.html,Collingwood,Bendigo Pioneers,Forward,3.0,50.0,2016,National Draft,Collingwood Magpies
1305,../players/K/Keegan_Brooksby.html,Hawthorn,South Adelaide,"Forward, Ruck",NaN,NaN,2020,SSP Recruit,None


In [761]:
temp_df = pd.DataFrame(list(extra_df[['afl_tables_url', 'origin', 'draft_round', 'draft_pick_number',
                                      'draft_year', 'draft_type', 'draft_team']].value_counts().index), 
                      columns=['afl_tables_url', 'origin', 'draft_round', 'draft_pick_number',
                                      'draft_year', 'draft_type', 'draft_team'])

In [784]:
temp_df2 = pd.DataFrame(list(extra_df[['afl_tables_url', 'team', 'position']].value_counts().index), 
                        columns=['afl_tables_url', 'team', 'position'])

In [787]:
player_df_final = player_df.merge(temp_df2, how='left', on=['afl_tables_url','team'])

In [794]:
player_data_df_final = player_data_df.merge(temp_df, how='left', on='afl_tables_url')

## Exporting

In [578]:
player_df.to_csv('afltables_stats')

In [180]:
player_agg_df.to_csv('afltables_agg_stats')

In [318]:
player_data_df.to_csv('afltables_player')

In [561]:
game_df.to_csv('afltables_game')

In [748]:
extra_df.to_csv('extra_bio_info')

In [796]:
player_data_df_final.to_csv('afltables_player_final')

In [788]:
player_df_final.to_csv('afltables_stats_final')

## Final dfs

In [789]:
game_df

,year,round,venue,home_team,away_team,attendance,home_team_final_score,away_team_final_score,day_of_week,date,...,home_Q1,home_Q2,home_Q3,home_Q4,home_extra_time,away_Q1,away_Q2,away_Q3,away_Q4,away_extra_time
0,2012,1,Stadium Australia,Greater Western Sydney,Sydney,"38,203",37.0,100.0,Sat,2012-03-24,...,1.2,3.3,3.4,5.7,None,4.1,8.4,13.8,14.16,None
1,2012,1,M.C.G.,Richmond,Carlton,"78,285",81.0,125.0,Thu,2012-03-29,...,3.4,5.6,10.7,12.9,None,3.2,8.7,11.13,18.17,None
2,2012,1,M.C.G.,Hawthorn,Collingwood,"78,466",137.0,115.0,Fri,2012-03-30,...,2.1,10.6,14.10,20.17,None,2.7,7.9,12.16,16.19,None
3,2012,1,M.C.G.,Melbourne,Brisbane Lions,"33,473",78.0,119.0,Sat,2012-03-31,...,3.3,7.4,8.8,11.12,None,1.4,7.8,13.13,17.17,None
4,2012,1,Carrara,Gold Coast,Adelaide,"12,790",68.0,137.0,Sat,2012-03-31,...,3.1,5.3,8.6,10.8,None,7.8,11.10,15.16,19.23,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2253,2023,3,M.C.G.,St Kilda,Essendon,"69,255",92.0,74.0,Sat,2023-04-01,...,5.4,7.5,9.7,14.8,None,1.1,4.5,6.6,11.8,None
2254,2023,3,Adelaide Oval,Port Adelaide,Adelaide,"48,962",86.0,117.0,Sat,2023-04-01,...,6.1,9.3,11.5,13.8,None,5.1,9.3,11.8,18.9,None
2255,2023,3,Carrara,Gold Coast,Geelong,"13,122",73.0,54.0,Sun,2023-04-02,...,2.3,3.9,6.11,10.13,None,3.4,3.6,5.10,7.12,None
2256,2023,3,M.C.G.,Melbourne,Sydney,"42,423",134.0,84.0,Sun,2023-04-02,...,6.1,9.3,14.4,21.8,None,1.3,6.6,9.9,12.12,None


In [798]:
player_data_df_final

,afl_tables_url,name,height,weight,dob,debut_date,last_date,origin,draft_round,draft_pick_number,draft_year,draft_type,draft_team
0,../players/J/James_Madden.html,James Madden,186,85,1999-11-15,2001-10-25,2000-08-16,NaN,NaN,NaN,NaN,NaN,NaN
1,../players/J/Jarrod_Lienert.html,Jarrod Lienert,195,88,1994-08-05,1999-04-25,1995-03-25,NaN,NaN,NaN,NaN,NaN,NaN
2,../players/S/Shane_Kersten.html,Shane Kersten,191,90,1993-03-15,2002-01-02,1997-10-29,South Fremantle,2.0,34.0,2011,National Draft,Geelong Cats
3,../players/M/Mitch_Hallahan.html,Mitch Hallahan,183,84,1992-08-23,2001-08-12,1998-04-07,Dandenong Stingrays,2.0,38.0,2010,National Draft,Hawthorn Hawks
4,../players/J/Jack_Lonie.html,Jack Lonie,175,70,1996-08-13,2004-08-18,1998-06-11,Dandenong Stingrays,3.0,41.0,2014,National Draft,St Kilda Saints
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,../players/J/Jed_Anderson.html,Jed Anderson,179,81,1994-02-02,2004-02-12,1994-09-30,NaN,NaN,NaN,NaN,NaN,NaN
1024,../players/W/Will_Walker.html,Will Walker,186,78,1999-03-30,2003-12-11,2001-11-23,Sandringham Dragons,2.0,23.0,2017,National Draft,North Melbourne Kangaroos
1025,../players/R/Reuben_Ginbey.html,Reuben Ginbey,191,83,2004-09-10,2004-10-03,2004-09-18,East Perth,1.0,9.0,2022,National Draft,West Coast Eagles
1026,../players/M/Mitch_McGovern.html,Mitch McGovern,191,93,1994-10-11,2001-10-25,1994-10-20,Claremont,3.0,43.0,2014,National Draft,Adelaide Crows


In [793]:
player_df_final

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,BR,CP,UP,CM,MI,1%,BO,GA,%P,position
0,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,1.0,Port Adelaide,7,W,42,4.0,...,0.0,10.0,2.0,0.0,0.0,3.0,0.0,0.0,78.0,Defender
1,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,2.0,Richmond,10,W,42,7.0,...,0.0,9.0,3.0,2.0,0.0,0.0,0.0,0.0,77.0,Defender
2,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,3.0,Greater Western Sydney,11,W,42,11.0,...,0.0,5.0,9.0,0.0,0.0,3.0,0.0,0.0,82.0,Defender
3,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,4.0,North Melbourne,14,W,42,9.0,...,0.0,6.0,7.0,0.0,0.0,6.0,0.0,0.0,74.0,Defender
4,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,5.0,Geelong,15,W,42,2.0,...,0.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,76.0,Defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52631,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,207.0,Hawthorn,23,W,11,13.0,...,0.0,8.0,19.0,0.0,0.0,2.0,0.0,1.0,79.0,Midfield
52632,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,208.0,Fremantle,EF,L,11,19.0,...,0.0,13.0,23.0,0.0,0.0,2.0,0.0,0.0,81.0,Midfield
52633,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,209.0,Melbourne,1,L,11,14.0,...,0.0,13.0,20.0,0.0,0.0,0.0,0.0,1.0,78.0,Midfield
52634,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,210.0,St Kilda,2,L,11,7.0,...,0.0,9.0,18.0,0.0,0.0,3.0,0.0,1.0,78.0,Midfield


In [791]:
player_data_df

,afl_tables_url,name,height,weight,dob,debut_date,last_date,origin,draft_round,draft_pick_number,draft_year,draft_type,draft_team
0,../players/J/James_Madden.html,James Madden,186,85,1999-11-15,2001-10-25,2000-08-16,NaN,NaN,NaN,NaN,NaN,NaN
1,../players/J/Jarrod_Lienert.html,Jarrod Lienert,195,88,1994-08-05,1999-04-25,1995-03-25,NaN,NaN,NaN,NaN,NaN,NaN
2,../players/S/Shane_Kersten.html,Shane Kersten,191,90,1993-03-15,2002-01-02,1997-10-29,South Fremantle,2.0,34.0,2011,National Draft,Geelong Cats
3,../players/M/Mitch_Hallahan.html,Mitch Hallahan,183,84,1992-08-23,2001-08-12,1998-04-07,Dandenong Stingrays,2.0,38.0,2010,National Draft,Hawthorn Hawks
4,../players/J/Jack_Lonie.html,Jack Lonie,175,70,1996-08-13,2004-08-18,1998-06-11,Dandenong Stingrays,3.0,41.0,2014,National Draft,St Kilda Saints
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,../players/J/Jed_Anderson.html,Jed Anderson,179,81,1994-02-02,2004-02-12,1994-09-30,NaN,NaN,NaN,NaN,NaN,NaN
1024,../players/W/Will_Walker.html,Will Walker,186,78,1999-03-30,2003-12-11,2001-11-23,Sandringham Dragons,2.0,23.0,2017,National Draft,North Melbourne Kangaroos
1025,../players/R/Reuben_Ginbey.html,Reuben Ginbey,191,83,2004-09-10,2004-10-03,2004-09-18,East Perth,1.0,9.0,2022,National Draft,West Coast Eagles
1026,../players/M/Mitch_McGovern.html,Mitch McGovern,191,93,1994-10-11,2001-10-25,1994-10-20,Claremont,3.0,43.0,2014,National Draft,Adelaide Crows


In [792]:
player_df

,name,afl_tables_url,team,year,game_num,opponent,Rd,R,#,KI,...,FA,BR,CP,UP,CM,MI,1%,BO,GA,%P
0,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,1.0,Port Adelaide,7,W,42,4.0,...,0.0,0.0,10.0,2.0,0.0,0.0,3.0,0.0,0.0,78.0
1,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,2.0,Richmond,10,W,42,7.0,...,1.0,0.0,9.0,3.0,2.0,0.0,0.0,0.0,0.0,77.0
2,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,3.0,Greater Western Sydney,11,W,42,11.0,...,0.0,0.0,5.0,9.0,0.0,0.0,3.0,0.0,0.0,82.0
3,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,4.0,North Melbourne,14,W,42,9.0,...,0.0,0.0,6.0,7.0,0.0,0.0,6.0,0.0,0.0,74.0
4,James Madden,../players/J/James_Madden.html,Brisbane Lions,2021,5.0,Geelong,15,W,42,2.0,...,0.0,0.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57293,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,207.0,Hawthorn,23,W,11,13.0,...,1.0,0.0,8.0,19.0,0.0,0.0,2.0,0.0,1.0,79.0
57294,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2022,208.0,Fremantle,EF,L,11,19.0,...,0.0,0.0,13.0,23.0,0.0,0.0,2.0,0.0,0.0,81.0
57296,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,209.0,Melbourne,1,L,11,14.0,...,1.0,0.0,13.0,20.0,0.0,0.0,0.0,0.0,1.0,78.0
57297,Jack Macrae,../players/J/Jack_Macrae.html,Western Bulldogs,2023,210.0,St Kilda,2,L,11,7.0,...,0.0,0.0,9.0,18.0,0.0,0.0,3.0,0.0,1.0,78.0
